<a href="https://colab.research.google.com/github/stevejobs1524/credify/blob/main/credify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install fastapi uvicorn requests torch numpy instagram-api nest-asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [11]:
!pip install instaloader


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.3 MB/s eta 0:00:00


In [14]:
!pip install fastapi uvicorn nest_asyncio


In [15]:
import nest_asyncio
nest_asyncio.apply()


In [16]:
import uvicorn
from fastapi import FastAPI

# Define FastAPI app (as you've already done)
app = FastAPI()

# Your existing FastAPI code...

# Run the server
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [1738]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1738]


In [12]:
import instaloader

def get_instagram_data(username):
    # Initialize Instaloader
    L = instaloader.Instaloader()

    try:
        # Load the profile
        profile = instaloader.Profile.from_username(L.context, username)

        # Extract relevant data
        data = {
            'followers_count': profile.followers,
            'following_count': profile.followees,
            'media_count': profile.mediacount,
            'bio': profile.biography,
            'is_private': profile.is_private,
        }

        return data
    except instaloader.exceptions.InstaloaderException as e:
        print(f"Error retrieving data for {username}: {e}")
        return None


In [17]:
# Install required packages
!pip install fastapi uvicorn nest_asyncio instaloader

import nest_asyncio
nest_asyncio.apply()

import instaloader
from fastapi import FastAPI, HTTPException
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Initialize FastAPI app
app = FastAPI()

# Define the GNN Model (same as before)
class FakeAccountGNN(nn.Module):
    def __init__(self, num_features):
        super(FakeAccountGNN, self).__init__()
        self.conv1 = GCNConv(num_features, 64)
        self.conv2 = GCNConv(64, 32)
        self.fc = nn.Linear(32, 1)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.fc(x)
        return torch.sigmoid(x)

# Dummy model loading and prediction
model = FakeAccountGNN(num_features=3)  # Adjust according to your model's needs

# Function to scrape Instagram data
def get_instagram_data(username):
    L = instaloader.Instaloader()

    try:
        profile = instaloader.Profile.from_username(L.context, username)
        data = {
            'followers_count': profile.followers,
            'following_count': profile.followees,
            'media_count': profile.mediacount,
            'bio': profile.biography,
            'is_private': profile.is_private,
        }
        return data
    except instaloader.exceptions.InstaloaderException:
        return None

# Define the request body
class UsernameRequest(BaseModel):
    username: str

# Endpoint for evaluating an account
@app.post("/evaluate")
async def evaluate_account(request: UsernameRequest):
    username = request.username

    # Get Instagram data
    data = get_instagram_data(username)

    if data is None:
        raise HTTPException(status_code=404, detail="Account not found or private.")

    # Preprocess data for the model
    features = torch.tensor([[data['followers_count'], data['media_count'], data['following_count']]], dtype=torch.float)

    # Dummy edge index
    edge_index = torch.tensor([[0, 0], [0, 0]], dtype=torch.long)

    # Evaluate using the model
    model.eval()
    with torch.no_grad():
        result = model(features, edge_index)

    prediction = "Fake" if result.item() > 0.5 else "Real"
    return {"username": username, "prediction": prediction}

# Run the FastAPI app
import uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [1738]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1738]
